In [1]:
from scipy.spatial.transform import Rotation as R
import numpy as np

In [2]:
r = [-0.15178555, 0.37758884, 0.27728027, 0.8703467]
t = [-0.14371057, -3.6296544, 2.1912692]

In [3]:
r = R.from_quat(r)
r = r.as_matrix()
r

array([[ 0.5610846 , -0.59728505,  0.57309218],
       [ 0.36803491,  0.80015358,  0.47360802],
       [-0.74144074, -0.05481624,  0.6687756 ]])

In [4]:
t = np.array(t).reshape((3,1))
t

array([[-0.14371057],
       [-3.6296544 ],
       [ 2.1912692 ]])

In [5]:
p = np.concatenate([r,t],1)
p

array([[ 0.5610846 , -0.59728505,  0.57309218, -0.14371057],
       [ 0.36803491,  0.80015358,  0.47360802, -3.6296544 ],
       [-0.74144074, -0.05481624,  0.6687756 ,  2.1912692 ]])

In [6]:
def rodrigues(r: np.ndarray) -> np.ndarray:
    '''
        if mat, convert to vec.
        if vec, convert to mat.
    '''
    if r.size == 3: return R.from_rotvec(r.squeeze()).as_matrix()
    elif r.size == 4: return R.from_quat(r.squeeze()).as_matrix()
    else: return R.from_matrix(r).as_rotvec().reshape((3, 1))
    
def get_rot(transform: np.ndarray) -> np.ndarray:
    '''
        args
        transform: camera pose vec or mat
        
        return
        rotational matrix
    '''
    if transform.size == 6:
        transform = transform.reshape((6, 1))
        return rodrigues(transform[:3])
    elif transform.shape == (3, 4) or transform.shape == (4, 4): #rotation matrix
        return transform[:3, :3]
    elif len(transform.shape) == 3: #rotation matrix (w/ batch)
        return transform[:, :3, :3]
    else:
        print(
            'Invalid shape of input transform: {}'.format(transform.shape))
        return None

def get_tr(transform: np.ndarray) -> np.ndarray:
    if transform.size == 6:
        transform = transform.reshape((6, 1))
        return transform[3:6].reshape((3, 1))
    elif transform.shape == (3, 4) or transform.shape == (4, 4):
        return transform[:3, 3].reshape((3, 1))
    elif len(transform.shape) == 3:
        return transform[:, :3, 3].T.reshape((3, -1))
    else:
        print(
            'Invalid shape of input transform: {}'.format(transform.shape))
        return None

def inverse_transform(transform: np.ndarray) -> np.ndarray:
    R, tr = get_rot(transform), get_tr(transform)
    R_inv = R.transpose()
    tr_inv = -R_inv.dot(tr)
    if transform.size == 6:
        r_inv = rodrigues(R_inv)
        return np.concatenate((r_inv, tr_inv), axis=0) # (6, 1) vector
    else:
        return np.concatenate((/media/keunmo/hdd2/dataset/replica_capture/apartment_1_slam1/sparse/hloc_sfm_spp_sg/sfmR_inv, tr_inv), axis=1) # (3, 4) matrix

In [7]:
inv_p = inverse_transform(p)
inv_p

array([[ 0.5610846 ,  0.36803491, -0.74144074,  3.04116958],
       [-0.59728505,  0.80015358, -0.05481624,  2.93856192],
       [ 0.57309218,  0.47360802,  0.6687756 ,  0.33592546]])

In [9]:
p

array([[ 0.5610846 , -0.59728505,  0.57309218, -0.14371057],
       [ 0.36803491,  0.80015358,  0.47360802, -3.6296544 ],
       [-0.74144074, -0.05481624,  0.6687756 ,  2.1912692 ]])

In [10]:
temp_p = np.eye(4)
temp_p[:3, :4] = p
temp_p

array([[ 0.5610846 , -0.59728505,  0.57309218, -0.14371057],
       [ 0.36803491,  0.80015358,  0.47360802, -3.6296544 ],
       [-0.74144074, -0.05481624,  0.6687756 ,  2.1912692 ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [12]:
temp_p_inv = np.linalg.inv(temp_p)
temp_p_inv[:3, :4]

array([[ 0.5610846 ,  0.36803491, -0.74144074,  3.04116958],
       [-0.59728505,  0.80015358, -0.05481624,  2.93856192],
       [ 0.57309218,  0.47360802,  0.6687756 ,  0.33592546]])

In [17]:
rodrigues(get_rot(inv_p))

array([[ 0.3174113 ],
       [-0.78960721],
       [-0.57984367]])

In [18]:
get_tr(inv_p)

array([[3.04116958],
       [2.93856192],
       [0.33592546]])

In [1]:
from pathlib import Path
import numpy as np
import cv2
import h5py

In [2]:
def list_h5_names(path):
    names = []
    with h5py.File(str(path), 'r', libver='latest') as fd:
        def visit_fn(_, obj):
            if isinstance(obj, h5py.Dataset):
                names.append(obj.parent.name.strip('/'))
        fd.visititems(visit_fn)
    return list(set(names))

In [8]:
list_h5_names("/home/keunmo/workspace/Hierarchical-Localization/datasets/replica_capture/apartment_1_cam_arr1/sparse/hloc_sfm_spp_sg2/features.h5")

['images/frame000022.jpg',
 'images/frame000031.jpg',
 'images/frame000035.jpg',
 'images/frame000013.jpg',
 'images/frame000038.jpg',
 'images/frame000024.jpg',
 'images/frame000023.jpg',
 'images/frame000025.jpg',
 'images/frame000012.jpg',
 'images/frame000014.jpg',
 'images/frame000016.jpg',
 'images/frame000019.jpg',
 'images/frame000037.jpg',
 'images/frame000039.jpg',
 'images/frame000020.jpg',
 'images/frame000021.jpg',
 'images/frame000000.jpg',
 'images/frame000027.jpg',
 'images/frame000036.jpg',
 'images/frame000017.jpg',
 'images/frame000032.jpg',
 'images/frame000034.jpg',
 'images/frame000010.jpg',
 'images/frame000002.jpg',
 'images/frame000018.jpg',
 'images/frame000008.jpg',
 'images/frame000029.jpg',
 'images/frame000011.jpg',
 'images/frame000009.jpg',
 'images/frame000028.jpg',
 'images/frame000030.jpg',
 'images/frame000001.jpg',
 'images/frame000015.jpg',
 'images/frame000004.jpg',
 'images/frame000006.jpg',
 'images/frame000007.jpg',
 'images/frame000026.jpg',
 

In [15]:
import plyfile
import numpy as np
from pathlib import Path

In [17]:
partial_ply = Path("/media/keunmo/hdd2/dataset/replica_capture/apartment_1_slam1/sparse/hloc_sfm_spp_sg/sfm/points3D_1.ply")
partial_pcd = plyfile.PlyData.read(partial_ply)

In [93]:
orig_sfm_path = Path("/media/keunmo/hdd2/dataset/replica_capture/apartment_1_slam1/sparse/hloc_sfm_spp_sg/sfm")
orig_sfm_path.parent / f'{orig_sfm_path.stem}_partial'

PosixPath('/media/keunmo/hdd2/dataset/replica_capture/apartment_1_slam1/sparse/hloc_sfm_spp_sg/sfm_partial')

In [20]:
partial_points = np.vstack([partial_pcd['vertex']['x'],
                                partial_pcd['vertex']['y'],
                                partial_pcd['vertex']['z']]).T

In [51]:
partial_points[:10]

array([[-0.19143343,  4.487793  , -1.1415539 ],
       [-2.587902  ,  5.976671  ,  0.00722652],
       [-0.12819563,  4.3815966 , -1.176512  ],
       [-2.5900111 ,  6.1493664 ,  0.87580246],
       [ 0.6392101 ,  6.481084  ,  1.0165186 ],
       [ 0.32375196,  6.838373  , -0.4189019 ],
       [ 0.6306458 ,  6.8455887 ,  0.49021006],
       [ 0.47407344,  6.847724  , -0.7105549 ],
       [-0.6575464 ,  6.649716  , -1.0219162 ],
       [-0.8282546 ,  6.8135166 , -0.5000301 ]], dtype=float32)

In [52]:
partial_points[0]

array([-0.19143343,  4.487793  , -1.1415539 ], dtype=float32)

In [27]:
from hloc.utils.read_write_model import (
        read_cameras_binary, read_images_binary, read_model, write_model,
        qvec2rotmat, read_images_text, read_cameras_text, write_points3D_binary, read_points3D_binary)

In [29]:
full_points3D= read_points3D_binary("/media/keunmo/hdd2/dataset/replica_capture/apartment_1_slam1/sparse/hloc_sfm_spp_sg/sfm/points3D.bin")

In [59]:
full_points3D

{10273: Point3D(id=10273, xyz=array([-2.50208332,  0.19455462,  0.38212323]), rgb=array([80, 72, 65]), error=array(0.10444713), image_ids=array([46, 44, 45]), point2D_idxs=array([681, 521, 607])),
 10272: Point3D(id=10272, xyz=array([-2.51576015,  0.93347678,  0.18487395]), rgb=array([76, 71, 69]), error=array(0.08182732), image_ids=array([45, 46, 44]), point2D_idxs=array([598, 654, 516])),
 10271: Point3D(id=10271, xyz=array([-2.49195117,  0.11755612,  0.41083312]), rgb=array([71, 64, 60]), error=array(1.3019117), image_ids=array([45, 46, 44, 46]), point2D_idxs=array([606, 675, 515, 685])),
 10270: Point3D(id=10270, xyz=array([-2.51207232,  0.65755374,  0.27375569]), rgb=array([80, 75, 72]), error=array(0.02877975), image_ids=array([45, 46, 44]), point2D_idxs=array([594, 657, 513])),
 10269: Point3D(id=10269, xyz=array([-2.49167853,  0.82689112,  0.21827796]), rgb=array([106,  97, 100]), error=array(0.48732037), image_ids=array([45, 47, 44, 46]), point2D_idxs=array([595, 622, 512, 653

In [36]:
for point in full_points3D.values():
    print(point.xyz)
    break

[-2.50208332  0.19455462  0.38212323]


In [90]:
list(full_points3D.items())[78]

(10149,
 Point3D(id=10149, xyz=array([-2.51759613,  0.28522912, -0.46200801]), rgb=array([104,  91,  92]), error=array(0.08061151), image_ids=array([44, 45, 46, 43, 47]), point2D_idxs=array([1259, 1412, 1533, 1156, 1537])))

In [86]:
point = list(full_points3D.values())[-1]
point

Point3D(id=9677, xyz=array([-1.31460702,  1.33760535, -0.53932574]), rgb=array([40, 36, 34]), error=array(0.07775138), image_ids=array([42, 43, 44, 41]), point2D_idxs=array([1076, 1231, 1358,  975]))

In [92]:
len(full_points3D)

10604

In [85]:
test_pts = partial_points[0]

for point in full_points3D.values():
    if np.allclose(test_pts, point.xyz, atol=1e-7, rtol=1e-7):
    # if np.equal(point.xyz, test_pts).all():
        print("found")
        print("err: ", np.linalg.norm(point.xyz - test_pts))
        print(f"id: {point.id}, xyz: {point.xyz}")
        print(f"image_ids: {point.image_ids}")
        break

found
err:  9.654664964764402e-08
id: 4087, xyz: [-0.19143343  4.48779287 -1.14155387]
image_ids: [11 12 13 14 15 10]


In [95]:
from tqdm import tqdm

In [103]:
def split_sfm(orig_sfm_path: Path, partial_ply: Path, partial_sfm_path = None):
    partial_pcd = plyfile.PlyData.read(partial_ply)
    partial_points = np.vstack([partial_pcd['vertex']['x'],
                                partial_pcd['vertex']['y'],
                                partial_pcd['vertex']['z']]).T
    cameras, images, points3D = read_model(orig_sfm_path.as_posix())
    partial_cameras = {}
    partial_images = {}
    partial_points3D = {}
    for point in tqdm(partial_points):
        for _, point3D in points3D.items():
            if np.allclose(point, point3D.xyz, atol=1e-7, rtol=1e-7):
                partial_points3D[point3D.id] = point3D
                for image_id in point3D.image_ids:
                    if image_id not in partial_images:
                        partial_images[image_id] = images[image_id]
                        if images[image_id].camera_id not in partial_cameras:
                            partial_cameras[images[image_id].camera_id] = cameras[images[image_id].camera_id]
                break
    
    # check if all the cameras and images are in the partial set
    if len(partial_points3D) != len(partial_points):
        print(f'Get partial points: {len(partial_points3D)} out of {len(partial_points)}')
    else:
        print(f'Get all {len(partial_points3D)} partial points')
    
    if partial_sfm_path is None:
        partial_sfm_path = (orig_sfm_path.parent / f'{orig_sfm_path.stem}_partial').as_posix()
    
    Path(partial_sfm_path).mkdir(parents=True, exist_ok=True)
    write_model(partial_cameras, partial_images, partial_points3D, partial_sfm_path)
    print(f'Write partial SfM to {partial_sfm_path}')

In [108]:
orig_sfm_path = Path("/media/keunmo/hdd2/dataset/replica_capture/apartment_1_slam1/sparse/hloc_sfm_spp_sg/sfm")
partial_ply = Path("/media/keunmo/hdd2/dataset/replica_capture/apartment_1_slam1/sparse/hloc_sfm_spp_sg/sfm/points3D_1.ply")
split_sfm(orig_sfm_path, partial_ply)

100%|██████████| 4448/4448 [14:04<00:00,  5.27it/s]

Get all 4448 partial points
Write partial SfM to /media/keunmo/hdd2/dataset/replica_capture/apartment_1_slam1/sparse/hloc_sfm_spp_sg/sfm_partial


In [105]:
cameras, images, points3D = read_model(orig_sfm_path.as_posix())

In [114]:
# use KDTree to find the closest point
from scipy.spatial import KDTree

def split_sfm(orig_sfm_path: Path, partial_ply: Path, partial_sfm_path = None, thresh=1e-7):
    partial_pcd = plyfile.PlyData.read(partial_ply)
    partial_points = np.vstack([partial_pcd['vertex']['x'],
                                partial_pcd['vertex']['y'],
                                partial_pcd['vertex']['z']]).T
    cameras, images, points3D = read_model(orig_sfm_path.as_posix())
    points3D = list(points3D.values())
    partial_cameras = {}
    partial_images = {}
    partial_points3D = {}
    full_points = np.array([point.xyz for point in points3D])
    tree = KDTree(full_points)
    for point in tqdm(partial_points):
        dist, ind = tree.query(point, k=1)
        if dist > thresh:
            continue
        partial_points3D[points3D[ind].id] = points3D[ind]
        for image_id in points3D[ind].image_ids:
            if image_id not in partial_images:
                partial_images[image_id] = images[image_id]
                if images[image_id].camera_id not in partial_cameras:
                    partial_cameras[images[image_id].camera_id] = cameras[images[image_id].camera_id]
    
    if len(partial_points3D) != len(partial_points):
        print(f'Get partial points: {len(partial_points3D)} out of {len(partial_points)}')
    else:
        print(f'Get all {len(partial_points3D)} partial points')
    
    if partial_sfm_path is None:
        partial_sfm_path = (orig_sfm_path.parent / f'{orig_sfm_path.stem}_partial').as_posix()
    
    Path(partial_sfm_path).mkdir(parents=True, exist_ok=True)
    write_model(partial_cameras, partial_images, partial_points3D, partial_sfm_path)
    print(f'Write partial SfM to {partial_sfm_path}')
    

In [116]:
orig_sfm_path = Path("/media/keunmo/hdd2/dataset/replica_capture/apartment_1_slam1/sparse/hloc_sfm_spp_sg/sfm")
partial_ply1 = Path("/media/keunmo/hdd2/dataset/replica_capture/apartment_1_slam1/sparse/hloc_sfm_spp_sg/sfm/points3D_1.ply")
partial_ply2 = Path("/media/keunmo/hdd2/dataset/replica_capture/apartment_1_slam1/sparse/hloc_sfm_spp_sg/sfm/points3D_2.ply")
partial_sfm_path1 = Path("/media/keunmo/hdd2/dataset/replica_capture/apartment_1_slam1/sparse/hloc_sfm_spp_sg/sfm_partial1")
partial_sfm_path2 = Path("/media/keunmo/hdd2/dataset/replica_capture/apartment_1_slam1/sparse/hloc_sfm_spp_sg/sfm_partial2")
split_sfm(orig_sfm_path, partial_ply1, partial_sfm_path1)
split_sfm(orig_sfm_path, partial_ply2, partial_sfm_path2)

100%|██████████| 4448/4448 [00:00<00:00, 26800.55it/s]


Get all 4448 partial points
Write partial SfM to /media/keunmo/hdd2/dataset/replica_capture/apartment_1_slam1/sparse/hloc_sfm_spp_sg/sfm_partial1


100%|██████████| 7374/7374 [00:00<00:00, 29434.75it/s]


Get all 7374 partial points
Write partial SfM to /media/keunmo/hdd2/dataset/replica_capture/apartment_1_slam1/sparse/hloc_sfm_spp_sg/sfm_partial2
